In [1096]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd
import numpy as np
from src import parse

/Users/andrew/src/otp-scheduler


In [1097]:
def sort_schedule(df):
    return df.sort_values(by=['zone', 'priority_rank', 'code']).reset_index(drop=True)

restaurants = parse.parse_from_restaurants_csv('../../examples/boston/20200405/restaurants.csv').set_index('code')
hospitals = sort_schedule(parse.parse_from_hospital_requests_csv('../../examples/boston/20200405/hospitals.csv')).set_index('name')
vip_hospitals = sort_schedule(parse.parse_from_hospital_requests_csv('../../examples/boston/20200405/vips.csv')).set_index('name')
vip_schedule = vip_hospitals.iloc[:, 4:]

# Create empty results frame
output = hospitals.copy()
output.iloc[:, 4:] = np.nan

rest_usage = restaurants.copy()
rest_usage.iloc[:, 2:] = 0

rest_cap = restaurants.copy()

# ASSUMPTIONS
MEAL_CAPACITY = 10000
MAX_DROPOFFS_PER_RUN = 4
MIN_MEALS_PER_RUN = 50

curr_capacity = MEAL_CAPACITY

total_hospitals = hospitals.shape[0]

### Idea #1

1. Go in order of hospital priority + who has least meals served so far (start with VIPs?)
2. find the order request that’s the most meals (or a request that’s in the biggest unserved cluster) that hasn’t yet been fulfilled. 
3. Assign a restaurant that can handle that restaurant and hopefully everything else in the cluster. 
4. Adjust meal counts for other hospitals that will be delivered to in that cluster. 
5. Update meals for all hospitals, capacity for that restaurant, and repeat. 

#### Stuff to track?

Restaurants
- Where am I assigned delivery? Which cluster am I going to? --> output/rest_usage
- How many meals left can I make? --> restaurants
- How many days left can I work? --> rest_usage

Hospitals
- How many orders are already assigned?
- 

In [1098]:
def day_from_meal(meal):
    return meal.replace('lunch', '').replace('breakfast', '').replace('dinner', '').replace('_', '')

def get_meal_log(schedule, is_vip):
    log = []
    for name, row in schedule.iterrows():
        for j, val in enumerate(row[4:]):
            if not pd.isnull(val):
                log.append([
                    name,
                    row['zone'],
                    row['priority_rank'],
                    is_vip,
                    schedule.columns[j + 4],
                    val, 
                    np.nan,
                    day_from_meal(schedule.columns[j + 4]),
                    False
                ])
    return pd.DataFrame(log, columns=['hospital', 'zone', 'priority_rank', 'vip', 'meal', 'quantity', 'restaurant', 'day', 'failed'])

vip_log = get_meal_log(vip_hospitals, True)
meal_log = get_meal_log(hospitals, False).append(vip_log)

In [1099]:
# meal_log

In [1100]:
# 1. Fill outstanding VIP meals
# 2. Fill out with LCD approach (select min, if tie take one with better priority)

In [1101]:
z = get_rest_cap_cluster_for_meal('lunch_wed')
z[(~z.index.isin(['PG', 'LD'])) & (z['lunch_wed'] > 100)]

meal_log[(meal_log['meal'] == 'lunch_mon') & (meal_log['zone'] != 3) & (meal_log['restaurant'].notnull())]['restaurant'].unique()

array([], dtype=float64)

In [1102]:
def get_oustanding_vip_meals():
    return meal_log[(meal_log['vip'] == True) & meal_log['restaurant'].isnull()]

def get_outstanding_hospitals_in_cluster_for_meal(meal, zone):
    return meal_log[(meal_log['meal'] == meal) & (meal_log['zone'] == zone) & meal_log['restaurant'].isnull()]

def get_hospitals_in_cluster_for_meal(meal, zone):
    return meal_log[(meal_log['meal'] == meal) & (meal_log['zone'] == zone)]

def is_restaurant_serving_same_day_other_meal(rest_code, meal):
    day = day_from_meal(meal)
    rest_serving = meal_log[(meal_log['restaurant'] == rest_code) & (meal_log['day'] == day) & meal_log['meal'] != meal]
    return not rest_serving.empty

def assign_hospital_order(row):
    global curr_capacity

    meal = row['meal']
    hospital = row['hospital']
    
    print('Assigning', hospital, meal)
    
    # Check to see if we've already done this (sanity)
    if not pd.isnull(row['restaurant']):
        print('We have already assigned this one!')
        return False
        
    hospitals_in_cluster = get_hospitals_in_cluster_for_meal(meal, row['zone'])

    # Restaurant capacity
    r_cap = get_rest_cap_cluster_for_meal(meal)
        
    # The amount have to serve
    target_quantity = row['quantity']
    
    if curr_capacity - target_quantity < 0:
        print('We cannot to afford this meal because of our global limit!', curr_capacity, target_quantity)
        return False
    
    print(hospital, meal, row['zone'], target_quantity)
        
#     def get_candidate():
#         # Try to serve other (bonus) hospitals alongside this hospital, up to the MAX_DROPOFFS_PER_RUN limit
#         other_count = min(MAX_DROPOFFS_PER_RUN - 1, others_in_cluster.shape[0])
#         while(other_count >= 0):
#             # Sum of the remaining hospitals quantity, up to MAX_DROPOFFS_PER_RUN - 1, SORTED BY HIGHEST QUANTITY
#             other_quantity = others_in_cluster.iloc[:other_count, :]['quantity'].sum()

#             # Try to serve these on the same run
#             bonus_quantity = target_quantity + other_quantity
            
#             if bonus_quantity + target_quantity > curr_capacity:
#                 other_count -= 1
#                 continue

#             # Find any restaurants that can serve the hospital + bonus
#             r_candidates = r_cap[r_cap[meal] >= bonus_quantity]
            
#             # Take the one that's closest to the bonus
#             r_candidates = r_candidates[r_candidates[meal] == r_candidates[meal].min()]

#             # Found one!
#             if not r_candidates.empty:
#                 return r_candidates, other_count
            
#             # Otherwise try again, with one fewer bonus restaurant
#             other_count -= 1
        
#         return None, None
    
    def find_restaurant():
        # Check if a restaurant is already serving the cluster
        restaurants_serving_cluster = hospitals_in_cluster[hospitals_in_cluster['restaurant'].notnull()]['restaurant']
        
        if not restaurants_serving_cluster.empty:
            # Filter out any that are already at the dropout limit
            rest_counter = Counter(restaurants_serving_cluster)
            rest_cluster_keys = rest_counter.keys()
            print('Checking for restaurants already in cluster', rest_counter)
            rest_with_room = [c[0] for c in rest_counter.most_common() if c[1] < MAX_DROPOFFS_PER_RUN]
            
            # Find restaurants with capacity
            candidates_with_quantity = []
            for r in rest_with_room:
                q = rest_cap.loc[r, meal]
                if q >= target_quantity:
                    print('Found an existing restaurant with capacity', r, q)
                    candidates_with_quantity.append(r)
                    if is_restaurant_serving_same_day_other_meal(r, meal):
                        print('Returning a restaurant with capacity, working on the same day', r, q)
                        return r
            if len(candidates_with_quantity):
                print('Returning a restaurant with capacity', r, q)
                return candidates_with_quantity[0]
        
        print('No suitable restaurants already working this cluster')
        # Couldn't find an appropriate restaurant already serving cluster
        
        restaurants_in_other_zones = meal_log[(meal_log['meal'] == meal) & (meal_log['zone'] != row['zone']) & (meal_log['restaurant'].notnull())]['restaurant'].unique()
        
        print('filtering our candidates with two lists', restaurants_serving_cluster, restaurants_in_other_zones)
        r_candidates = r_cap[(~r_cap.index.isin(restaurants_serving_cluster)) & (~r_cap.index.isin(restaurants_in_other_zones)) & (r_cap[meal] >= target_quantity)]
        
        print('Number of candidates with capacity', r_candidates.shape[0])
        
        if r_candidates.empty:
            return None
        
        # Sort in quantity order
        r_candidates = r_candidates.sort_values(by=meal, ascending=False)
        
        day = day_from_meal(meal)
        
        # TODO Make sure restaurant isn't overbooked on their max day count
        for i, r in r_candidates.iterrows():
            print(i)
            days_worked = set(meal_log[meal_log['restaurant'] == i]['day'])
            # Check if we're already working this day, or the new day is still under cap
            if day in days_worked or len(days_worked) < rest_cap.loc[i, 'max_days_per_week']:
                return i
        
        return None
          
    r_code = find_restaurant()
    
    if r_code is None:
        print('CANNOT SERVE HOSPITAL FOR THEIR REQUEST', hospital, meal, r_cap)
        return False
                
    # Allocate the restaurant to the hospital(s)
    print('debiting r capacity: ', r_code, rest_cap.loc[r_code, meal])
    rest_cap.loc[r_code, meal] -= target_quantity
            
    print('increasing r usage', r_code, target_quantity)
    rest_usage.loc[r_code, meal] += target_quantity
        
    curr_capacity -= target_quantity
                
    print('updating log', hospital, meal, r_code)
    meal_log.loc[(meal_log['hospital'] == hospital) & (meal_log['meal'] == meal), 'restaurant'] = r_code
    
    return True
    
#     print('updating remaining assignments', other_count)
#     other_amt = 0
#     for idx in range(other_count):
#         h = others_in_cluster.iloc[idx]
#         print('\t', h['hospital'], h['quantity'])
#         rest_usage.loc[r_code, meal] += h['quantity']
#         curr_capacity -= h['quantity']
#         other_amt += h['quantity']
#         meal_log.loc[(meal_log['hospital'] == h['hospital']) & (meal_log['meal'] == meal), 'restaurant'] = r_code
#     print('other amount assigned', other_amt)
#     print('total', other_amt + target_quantity)
    
for i, row in get_oustanding_vip_meals().iterrows():
    assigned = assign_hospital_order(row)
    if not assigned:
        meal_log.loc[row['hospital'], 'failed'] = True
    print()
    print()
    

Assigning Brigham and Women's Hospital - ED lunch_wed
Brigham and Women's Hospital - ED lunch_wed 1 100.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: float64) []
Number of candidates with capacity 6
SU
debiting r capacity:  SU 1000.0
increasing r usage SU 100.0
updating log Brigham and Women's Hospital - ED lunch_wed SU


Assigning Brigham and Women's Hospital - ED lunch_sun
Brigham and Women's Hospital - ED lunch_sun 1 100.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: object) []
Number of candidates with capacity 5
SU
debiting r capacity:  SU 1000.0
increasing r usage SU 100.0
updating log Brigham and Women's Hospital - ED lunch_sun SU


Assigning Brigham and Women's Hospital - ED dinner_wed
Brigham and Women's Hospital - ED dinner_wed 1 100.0
No suitable restaurants already working this cluster
filtering our candidat

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)



Assigning Brigham and Women's - Faulkner ED lunch_wed
Brigham and Women's - Faulkner ED lunch_wed 2 45.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: object) ['SU']
Number of candidates with capacity 5
LD
debiting r capacity:  LD 700.0
increasing r usage LD 45.0
updating log Brigham and Women's - Faulkner ED lunch_wed LD


Assigning Brigham and Women's - Faulkner ED lunch_sun
Brigham and Women's - Faulkner ED lunch_sun 2 45.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: object) ['SU']
Number of candidates with capacity 4
LD
debiting r capacity:  LD 700.0
increasing r usage LD 45.0
updating log Brigham and Women's - Faulkner ED lunch_sun LD


Assigning Brigham and Women's - Faulkner ED dinner_wed
Brigham and Women's - Faulkner ED dinner_wed 2 45.0
No suitable restaurants already working this cluster
filtering our candida

debiting r capacity:  SU 1000.0
increasing r usage SU 12.0
updating log Boston Healthcare for the Homeless dinner_fri SU


Assigning Boston Healthcare for the Homeless dinner_sat
Boston Healthcare for the Homeless dinner_sat 2 12.0
Checking for restaurants already in cluster Counter({'SU': 1})
Found an existing restaurant with capacity SU 700.0
Returning a restaurant with capacity, working on the same day SU 700.0
debiting r capacity:  SU 700.0
increasing r usage SU 12.0
updating log Boston Healthcare for the Homeless dinner_sat SU


Assigning Boston Healthcare for the Homeless dinner_sun
Boston Healthcare for the Homeless dinner_sun 2 12.0
Checking for restaurants already in cluster Counter({'PG': 2})
Found an existing restaurant with capacity PG 155.0
Returning a restaurant with capacity, working on the same day PG 155.0
debiting r capacity:  PG 155.0
increasing r usage PG 12.0
updating log Boston Healthcare for the Homeless dinner_sun PG




In [1103]:
meal_log[meal_log.meal == 'lunch_mon']

,hospital,zone,priority_rank,vip,meal,quantity,restaurant,day,failed
0,BIDMC - West Campus,1,1,False,lunch_mon,60.0,NaN,mon,False
7,BIDMC - East Campus,1,1,False,lunch_mon,30.0,NaN,mon,False
23,Dana Farber Cancer Institute - Inpatient PAs,1,1,False,lunch_mon,30.0,NaN,mon,False
30,Whittier Street Health Center,1,2,False,lunch_mon,150.0,NaN,mon,False
48,Southern Jamaica Plain Health Center (part of ...,2,1,False,lunch_mon,30.0,NaN,mon,False
60,BWH - Faulkner 7N Covid Unit,2,1,False,lunch_mon,18.0,NaN,mon,False
74,BWH - Faulkner ICU,2,1,False,lunch_mon,17.0,NaN,mon,False
88,Mattapan Community Health Center,2,2,False,lunch_mon,35.0,NaN,mon,False
91,Dot House Health Dorchester,2,2,False,lunch_mon,25.0,NaN,mon,False
98,Upham's Corner Health Center,2,2,False,lunch_mon,70.0,NaN,mon,False


In [1104]:
hospitals_that_failed_placement = set()

def select_and_assign_hospital():
    next_hospital = meal_log[~meal_log.hospital.isin(list(hospitals_that_failed_placement))].groupby('hospital')[['priority_rank', 'restaurant']].agg({
            'restaurant': 'count',
            'priority_rank': 'min'
        }).sort_values(by=['restaurant', 'priority_rank']).reset_index().iloc[0]
    
    print('Next hospital:', next_hospital)

    # Find potential orders to serve, ordered by quantity
    possible_orders = meal_log[(meal_log['hospital'] == next_hospital['hospital']) & (meal_log['restaurant'].isnull()) & (meal_log['failed'] == False)].sort_values(by='quantity', ascending=False)
    
    print('Possible orders', possible_orders)
    for i, row in possible_orders.iterrows():
        if assign_hospital_order(row):
            return True
        else:
            meal_log.loc[
                (meal_log['hospital'] == row['hospital']) & (meal_log['meal'] == row['meal']), 
                'failed'
            ] = True
            
    hospitals_that_failed_placement.add(next_hospital['hospital'])
    

while curr_capacity > 0 and len(hospitals_that_failed_placement) < total_hospitals:
    select_and_assign_hospital()
    print()
    print()

Next hospital: hospital         BIDMC - East Campus
restaurant                         0
priority_rank                      1
Name: 0, dtype: object
Possible orders                hospital  zone  priority_rank    vip         meal  quantity  \
7   BIDMC - East Campus     1              1  False    lunch_mon      30.0   
8   BIDMC - East Campus     1              1  False   lunch_tues      30.0   
9   BIDMC - East Campus     1              1  False    lunch_wed      30.0   
10  BIDMC - East Campus     1              1  False  lunch_thurs      30.0   
11  BIDMC - East Campus     1              1  False    lunch_fri      30.0   
12  BIDMC - East Campus     1              1  False    lunch_sat       6.0   
13  BIDMC - East Campus     1              1  False    lunch_sun       6.0   

   restaurant    day  failed  
7         NaN    mon   False  
8         NaN   tues   False  
9         NaN    wed   False  
10        NaN  thurs   False  
11        NaN    fri   False  
12        NaN    sat   F

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_value


Assigning Boston Children's Hospital - ED dinner_mon
Boston Children's Hospital - ED dinner_mon 1 60.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: object) ['SU']
Number of candidates with capacity 3
PG
debiting r capacity:  PG 500.0
increasing r usage PG 60.0
updating log Boston Children's Hospital - ED dinner_mon PG


Next hospital: hospital         Cambridge Health Alliance - Cambridge Hospital
restaurant                                                    0
priority_rank                                                 1
Name: 0, dtype: object
Possible orders                                            hospital  zone  priority_rank  \
114  Cambridge Health Alliance - Cambridge Hospital     3              1   
115  Cambridge Health Alliance - Cambridge Hospital     3              1   
116  Cambridge Health Alliance - Cambridge Hospital     3              1   
117  Cambridge Health Alliance - Cambridge 

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_value

 hospital         Southern Jamaica Plain Health Center (part of ...
restaurant                                                       0
priority_rank                                                    1
Name: 0, dtype: object
Possible orders                                              hospital  zone  priority_rank  \
48  Southern Jamaica Plain Health Center (part of ...     2              1   
49  Southern Jamaica Plain Health Center (part of ...     2              1   
50  Southern Jamaica Plain Health Center (part of ...     2              1   
51  Southern Jamaica Plain Health Center (part of ...     2              1   
52  Southern Jamaica Plain Health Center (part of ...     2              1   

      vip         meal  quantity restaurant    day  failed  
48  False    lunch_mon      30.0        NaN    mon   False  
49  False   lunch_tues      30.0        NaN   tues   False  
50  False    lunch_wed      30.0        NaN    wed   False  
51  False  lunch_thurs      30.0        NaN  t

...

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_value


Next hospital: hospital         Codman Square Health Center
restaurant                                 3
priority_rank                              2
Name: 0, dtype: object
Possible orders                         hospital  zone  priority_rank    vip         meal  \
105  Codman Square Health Center     2              2  False  lunch_thurs   
106  Codman Square Health Center     2              2  False    lunch_fri   

     quantity restaurant    day  failed  
105     110.0        NaN  thurs   False  
106     110.0        NaN    fri   False  
Assigning Codman Square Health Center lunch_thurs
We cannot to afford this meal because of our global limit! 16.0 110.0
Assigning Codman Square Health Center lunch_fri
We cannot to afford this meal because of our global limit! 16.0 110.0


Next hospital: hospital         Dot House Health Dorchester
restaurant                                 3
priority_rank                              2
Name: 0, dtype: object
Possible orders                        

Assigning BWH - Faulkner 7N Covid Unit lunch_fri
We cannot to afford this meal because of our global limit! 10.0 18.0
Assigning BWH - Faulkner 7N Covid Unit lunch_sat
We cannot to afford this meal because of our global limit! 10.0 18.0
Assigning BWH - Faulkner 7N Covid Unit lunch_sun
We cannot to afford this meal because of our global limit! 10.0 18.0


Next hospital: hospital         BWH - Faulkner ICU
restaurant                        4
priority_rank                     1
Name: 0, dtype: object
Possible orders               hospital  zone  priority_rank    vip          meal  quantity  \
78  BWH - Faulkner ICU     2              1  False     lunch_fri      17.0   
79  BWH - Faulkner ICU     2              1  False     lunch_sat      17.0   
80  BWH - Faulkner ICU     2              1  False     lunch_sun      17.0   
81  BWH - Faulkner ICU     2              1  False    dinner_mon      15.0   
82  BWH - Faulkner ICU     2              1  False   dinner_tues      15.0   
83  BWH - Faul

Assigning Carney Hospital - ED dinner_sun
We cannot to afford this meal because of our global limit! 10.0 100.0


Next hospital: hospital         Dana Farber Cancer Institute - Inpatient PAs
restaurant                                                  4
priority_rank                                               1
Name: 0, dtype: object
Possible orders                                         hospital  zone  priority_rank    vip  \
27  Dana Farber Cancer Institute - Inpatient PAs     1              1  False   
28  Dana Farber Cancer Institute - Inpatient PAs     1              1  False   
29  Dana Farber Cancer Institute - Inpatient PAs     1              1  False   

         meal  quantity restaurant  day  failed  
27  lunch_fri      30.0        NaN  fri   False  
28  lunch_sat      20.0        NaN  sat   False  
29  lunch_sun      20.0        NaN  sun   False  
Assigning Dana Farber Cancer Institute - Inpatient PAs lunch_fri
We cannot to afford this meal because of our global limit! 1

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [1105]:
curr_capacity, hospitals_that_failed_placement

(4.0,
 {'BIDMC - Chelsea',
  'BIDMC - East Campus',
  'BIDMC - ICU East',
  'BIDMC - ICU West',
  'BIDMC - West Campus',
  'BWH - Faulkner 7N Covid Unit',
  'BWH - Faulkner ICU',
  "Boston Children's Hospital - ED",
  'Boston Healthcare for the Homeless',
  'Boston Medical Center',
  "Brigham and Women's - Faulkner ED",
  "Brigham and Women's Hospital - ED",
  'Cambridge Health Alliance - Cambridge Hospital',
  'Cambridge Health Alliance - Everett Hospital',
  'Cambridge Health Alliance - Somerville Hospital',
  'Carney Hospital - ED',
  'Charles River Community Health',
  'Codman Square Health Center',
  'Dana Farber Cancer Institute - Inpatient PAs',
  'Dot House Health Dorchester',
  'East Boston Neighborhood Health Center',
  'Massachusetts General Hospital - ED',
  'Massachusetts General Hospital - ICU',
  'Mattapan Community Health Center',
  'South Boston Community Health Center',
  "Southern Jamaica Plain Health Center (part of Brigham & Women's)",
  'Tufts Medical Center',
  "

In [1106]:
meal_log.groupby('hospital')[['priority_rank', 'restaurant']].agg({
            'restaurant': 'count',
            'priority_rank': 'min'
        }).sort_values(by=['restaurant', 'priority_rank'])

,restaurant,priority_rank
hospital,,
BIDMC - ICU East,2,1
BIDMC - ICU West,2,1
Massachusetts General Hospital - ICU,2,1
Charles River Community Health,3,2
Codman Square Health Center,3,2
Dot House Health Dorchester,3,2
East Boston Neighborhood Health Center,3,2
Mattapan Community Health Center,3,2
South Boston Community Health Center,3,2


In [1107]:
# for i, row in meal_log.iterrows():
#     if not pd.isnull(row['restaurant']):
#         val = str(row['restaurant'] + ' - ' + str(row['quantity']))
#     else:
#         val = str('XXX - ' + str(row['quantity']))
#     output.loc[row['hospital'], row['meal']] = val
    
for i, row in meal_log.iterrows():
    if not pd.isnull(row['restaurant']):
        output.loc[row['hospital'], row['meal']] = row['restaurant']
        
output.to_csv('../../output.csv')

In [986]:
rest_usage

,name,max_days_per_week,breakfast_mon,breakfast_tues,breakfast_wed,breakfast_thurs,breakfast_fri,breakfast_sat,breakfast_sun,lunch_mon,...,lunch_fri,lunch_sat,lunch_sun,dinner_mon,dinner_tues,dinner_wed,dinner_thurs,dinner_fri,dinner_sat,dinner_sun
code,,,,,,,,,,,,,,,,,,,,,
PG,Pagu,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,274.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LD,Little Donkey,7,12.0,12.0,12.0,12.0,82.0,82.0,12.0,585.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MM,Mei Mei Restaurant,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PO,Porto,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FC,Flour Cafe,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CS,Cafe Sushi,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SU,Suya,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,729.0,...,647.0,148.0,243.0,405.0,485.0,470.0,405.0,12.0,392.0,537.0
IG,Iggy's Bakery,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1093]:
rest_usage.iloc[:, 2:].sum(axis=1).sort_values(ascending=False)

code
SU    4224.0
LD    2719.0
PG    2011.0
FC     383.0
PO     349.0
IG     310.0
CS       0.0
MM       0.0
dtype: float64

In [1094]:
meal_log.groupby(['priority_rank', 'hospital']).count()

zone  vip  \
priority_rank hospital                                                        
1             BIDMC - East Campus                                    7    7   
              BIDMC - ICU East                                       2    2   
              BIDMC - ICU West                                       2    2   
              BIDMC - West Campus                                    7    7   
              BWH - Faulkner 7N Covid Unit                          14   14   
              BWH - Faulkner ICU                                    14   14   
              Boston Children's Hospital - ED                        5    5   
              Boston Medical Center                                 14   14   
              Brigham and Women's - Faulkner ED                      4    4   
              Brigham and Women's Hospital - ED                      4    4   
              Cambridge Health Alliance - Cambridge Hospital         7    7   
              Cambridge Health Alliance - Everett Hospital           7    7   
              Cambridge Health Alliance - Somerville Hospital        7    7   
              Carney Hospital - ED                                   7    7   
              Dana Farber Cancer Institute - Inpatient PAs           7    7   
              Massachusetts General Hospital - ED                    5    5   
              Massachusetts General Hospital - ICU                   2    2   
              Southern Jamaica Plain Health Center (part of B...     5    5   
              Tufts Medical Center                                  10   10   
              VA Medical Center - West Roxbury                       6    6   
2             BIDMC - Chelsea                                        5    5   
              Boston Healthcare for the Homeless                    21   21   
              Charles River Community Health                         5    5   
              Codman Square Health Center                            5    5   
              Dot House Health Dorchester                            5    5   
              East Boston Neighborhood Health Center                 6    6   
              Mattapan Community Health Center                       3    3   
              South Boston Community Health Center                   5    5   
              Upham's Corner Health Center                           6    6   
              Whittier Street Health Center                          5    5   

                                                                  meal  \
priority_rank hospital                                                   
1             BIDMC - East Campus                                    7   
              BIDMC - ICU East                                       2   
              BIDMC - ICU West                                       2   
              BIDMC - West Campus                                    7   
              BWH - Faulkner 7N Covid Unit                          14   
              BWH - Faulkner ICU                                    14   
              Boston Children's Hospital - ED                        5   
              Boston Medical Center                                 14   
              Brigham and Women's - Faulkner ED                      4   
              Brigham and Women's Hospital - ED                      4   
              Cambridge Health Alliance - Cambridge Hospital         7   
              Cambridge Health Alliance - Everett Hospital           7   
              Cambridge Health Alliance - Somerville Hospital        7   
              Carney Hospital - ED                                   7   
              Dana Farber Cancer Institute - Inpatient PAs           7   
              Massachusetts General Hospital - ED                    5   
              Massachusetts General Hospital - ICU                   2   
              Southern Jamaica Plain Health Center (part of B...     5   
              Tufts Medical Center                       

In [542]:
def maybe_compact(df, meal, compact):
    if compact:
        return df[df[meal].notnull()]
    return df

def get_rest_cap_cluster_for_meal(meal, compact=False):
    r = rest_cap[[meal]]
    return maybe_compact(r, meal, compact)

def get_rest_usage_cluster_for_meal(meal):
    return rest_usage[[meal]]

head_columns = hospitals.columns[:4]
def get_hosp_cluster_for_meal(meal, zone, compact=False):
    h = hospitals.loc[:, [*head_columns, meal]]
    h = h[h['zone']==zone]
    return maybe_compact(h, meal, compact)

def get_output_cluster_for_meal(meal):
    return output.loc[:, [*head_columns, meal]]

In [543]:
def get_candidate_restaurants(meal, hospital, cluster):
    quantity = cluster[cluster.index == hospital][meal]
    
    # remove self from other hospitals in the cluster, along with any that already have assignments
    other_hosps = cluster[(cluster.index != hospital) & (cluster[meal].isnull())]

    # get remaining quantity from other hospitals
    other_quantity = other_hosps[meal].sum()

    # find potential restaurants *just for the hospital at hand*
    r_candidate = r_cap[r_cap[meal] >= quantity]

In [544]:
def assign_restaurant_to_cluster(meal, restaurant, hospital, hosps_cluster, rest_cap, rest_usage, output):
    pass

In [545]:
# Assign VIP meals
vip_schedule_dense = vip_schedule.dropna(how='all', axis=0).dropna(how='all', axis=1)
vip_schedule_dense
for i, row in vip_schedule_dense.iterrows():
    for j, val in enumerate(row):
        if not pd.isnull(val):
            hospital = hospitals.index.values[i]
            meal = vip_schedule_dense.columns[j]
            zone = hospitals['zone'][i]
            quantity = schedule.iloc[i, j]
            r_cap = get_rest_cap_cluster_for_meal(meal, compact=True)
            r_usage = get_rest_usage_cluster_for_meal(meal)
            cluster = get_hosp_cluster_for_meal(meal, zone, compact=True)
            
            # remove self from other hospitals in the cluster
            other_hosps = cluster[cluster.index != hospital]
            
            # get remaining quantity from other hospitals
            other_quantity = other_hosps[meal].sum()
            
            # find potential restaurants *just for the hospital at hand*
            # TODO make this smarter to encompass other restaurants above
            r_candidate = r_cap[r_cap[meal] >= quantity]
            
            # Do the assignment
            if not r_candidate.empty:
                # Take the restaurant with the most capacity
                highest_cap = r_candidate[r_candidate[meal] == r_candidate[meal].max()]
                rest_code = highest_cap.index.values[0]
                
                print('debiting r capacity to 0 - cannot serve this meal twice', r_cap[meal])
                rest_cap.loc[rest_code, meal] = 0
            
                print('increasing r usage', rest_code, meal, rest_usage.loc[rest_code, meal], quantity)
                rest_usage.loc[rest_code, meal] = quantity
                
                print('updating output', hospital, meal, rest_code)
                output.loc[hospital, meal] = rest_code
                
                # Get this restaurant to go to other hospitals in this cluster
                # ignore for now
                if not hosps.empty:
                    pass

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [316]:
output

,code,priority_rank,region,zone,breakfast_mon,breakfast_tues,breakfast_wed,breakfast_thurs,breakfast_fri,breakfast_sat,...,lunch_fri,lunch_sat,lunch_sun,dinner_mon,dinner_tues,dinner_wed,dinner_thurs,dinner_fri,dinner_sat,dinner_sun
name,,,,,,,,,,,,,,,,,,,,,
BIDMC - West Campus,2,1,Longwood,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BIDMC - East Campus,3,1,Longwood,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brigham and Women's Hospital - ED,4,1,Longwood,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,SU,NaN,NaN,SU,NaN,NaN,NaN,PG
Boston Children's Hospital - ED,18,1,Longwood,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BIDMC - ICU West,21,1,Longwood,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BIDMC - ICU East,22,1,Longwood,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dana Farber Cancer Institute - Inpatient PAs,23,1,Longwood,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Whittier Street Health Center,15,2,Longwood,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brigham and Women's - Faulkner ED,5,1,Roxbury / Jamaica Plain,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,LD,NaN,NaN,PG,NaN,NaN,NaN,SU


In [317]:
rest_cap.loc['LD'], output['lunch_wed']

(name                 Little Donkey
 max_days_per_week                7
 breakfast_mon                    0
 breakfast_tues                   0
 breakfast_wed                    0
 breakfast_thurs                  0
 breakfast_fri                    0
 breakfast_sat                    0
 breakfast_sun                    0
 lunch_mon                       88
 lunch_tues                      88
 lunch_wed                        0
 lunch_thurs                     88
 lunch_fri                       88
 lunch_sat                      700
 lunch_sun                        0
 dinner_mon                     500
 dinner_tues                    500
 dinner_wed                       0
 dinner_thurs                   500
 dinner_fri                     500
 dinner_sat                       0
 dinner_sun                       0
 Name: LD, dtype: object,
 name
 BIDMC - West Campus                                                 NaN
 BIDMC - East Campus                                               

code
PG       0.0
LD       0.0
MM       0.0
PO       NaN
FC       NaN
CS       NaN
SU    1000.0
IG       NaN
Name: dinner_sun, dtype: float64